In [1]:
import numpy as np
import pybullet as p
import pybullet_data
import matplotlib.pyplot as plt

import time

from utils import *
from casadi import *
import crocoddyl
from costs import *
from ocp_sys import QuadcopterCasadi, ActionDataRobot, ActionModelRobot
from IPython.display import clear_output
from ocp import ILQR_Standard, get_ilqr_from_ddp
from mixture_model import Gaussian
np.set_printoptions(precision=3, suppress=True)
%load_ext autoreload
%autoreload 2

### Configure pybullet

In [2]:
# configure pybullet and load plane.urdf and quadcopter.urdf
physicsClient = p.connect(p.GUI)  # pybullet only for computations no visualisation
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [3]:
p.resetSimulation()

#### Load robot model

In [4]:
#planeId = p.loadURDF("plane.urdf",[0,0,0],p.getQuaternionFromEuler([0,0,0]))
robot_urdf = "data/frankaemika_new/panda_arm_rivet.urdf"
#robot_urdf = "data/panda_arm_gripper.urdf"
robotId = p.loadURDF(robot_urdf)

# for i in range((p.getNumJoints(robotId))+1):
#     p.changeVisualShape(robotId, i-1, rgbaColor=[1,0,0,0.6])

In [5]:
from utils import set_q, vis_traj
from functools import partial

In [6]:
set_q_std = partial(set_q, robot_id = robotId, joint_indices=np.arange(7), set_base = False)

In [7]:
vis_traj_std = partial(vis_traj, vis_func = set_q_std)

In [8]:
_,_,target_id = create_primitives(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1], rgbaColor=(0,0,1,1))

#### Load robot in pinocchio

In [9]:
import pinocchio as pin
from functools import partial

In [10]:
rmodel_pin = pin.buildModelFromUrdf(robot_urdf)
rdata_pin = rmodel_pin.createData()

In [11]:
endEffector = 'panda_grasptarget_hand'
####Pinocchio indices####
pin_end_effector_id = rmodel_pin.getFrameId(endEffector)
pin_joint_names = [n for n in rmodel_pin.names]
computePoseStd = partial(computePose, rmodel_pin, rdata_pin, pin_end_effector_id)

#### Load obstacle

In [12]:
pos_obs_set = [np.array([0.4, .7, .2]), np.array([0., .7, .9]), np.array([-0.4, .7, .2])]
r_body = 0.2
r_obs = 0.15
margin = 0.1
w_obstacle = 1000

ballIds = []
cost_collisions = []
for pos_obs in pos_obs_set:
    _,_,ballid = create_primitives(radius = r_obs)
    p.resetBasePositionAndOrientation(ballid, pos_obs, (0,0,0,1))
    ballIds += [ballid]
    col_activation = ActivationCollision(nr = 3, threshold = r_body + r_obs + margin)
    cost_collision_i = RobotSphereCollisionCost(col_activation, nu = 4, rmodel = rmodel_pin, rdata = rdata_pin, ee_id = pin_end_effector_id, r_body = r_body, r_obs = r_obs, pos_obs = pos_obs, w = w_obstacle)
    cost_collisions += [cost_collision_i]

#### Define the robot_sys

In [13]:
from ocp_sys import Manipulator

In [14]:
sys_name = 'manipulator'
robot_sys = Manipulator(rmodel_pin, pin_end_effector_id)
T = 150
dt = 0.05
T_short = 30
n_iter_short = 5
use_collision = True

In [15]:
cost_collision = CostModelSum(robot_sys, cost_collisions)

In [16]:
q0 = np.array([0.3,0.,0.3,-1.6,0.2,1.7,0])
x0 = np.concatenate([q0, np.zeros(7)])
set_q_std(q0)

# 1. Plan using iLQR with long horizon

#### Setting cost

In [17]:
if sys_name == 'quadcopter':
    x_ref = np.array([0,6.,2.,0,0,0,0,0,0,0,0,0]) #for quadcopter
    p.resetBasePositionAndOrientation(target_id, x_ref[:3], (0,0,0,1))
    Q = np.eye(robot_sys.Dx)*.1
    Q[6:9,6:9]*= 1.
    Qf = np.eye(robot_sys.Dx)*1000
    R = np.eye(robot_sys.Du)*.1
elif sys_name == 'bicopter':
    x_ref = np.array([3,3,0,0,0,0]) #for bicopter
    Q = np.eye(robot_sys.Dx)*.1
    #Q[2:,2:] = 0*Q[2:,2:] 
    Qf = np.eye(robot_sys.Dx)*100
    #Qf[2:,2:] = 0*Qf[2:,2:] 
    R = np.eye(robot_sys.Du)*.05
elif sys_name == 'twolinkrobot':
    x_ref = np.array([np.pi/1.3,np.pi/1.2,0,0]) #for 2drobot
    Q = np.eye(robot_sys.Dx)*.1
    Qf = np.eye(robot_sys.Dx)*300
    R = np.eye(robot_sys.Du)*2
    
elif sys_name == 'manipulator':
    x_ref = x0.copy()#np.zeros(robot_sys.Dx) #for manipulator
    p_waypoint1 = np.array([0.4, .7, .8])
    p_waypoint2 = np.array([0., .7, .1])
    T_waypoint1 = 60
    T_waypoint2 = 110
    p_ref = np.array([-0.4, 0.7, 0.7])
    p.resetBasePositionAndOrientation(target_id, p_ref, (0,0,0,1))
    W = np.eye(3)*0.1
    WT = np.eye(3)*300
    Q = np.eye(robot_sys.Dx)*0.1
    Qf = np.eye(robot_sys.Dx)*0.1
    R = np.eye(robot_sys.Du)*2
    
costs = []
for i in range(T):
    if i <= T_waypoint1:
        p_ref_cur = p_waypoint1.copy()
    elif i <= T_waypoint2:
        p_ref_cur = p_waypoint2.copy()
    else:
        p_ref_cur = p_ref.copy()
        
    if i==T_waypoint1 or i == T_waypoint2:
        W_cur = WT.copy()
    else:
        W_cur = W.copy()
        
    runningStateCost = CostModelQuadratic(robot_sys, Q, x_ref = x_ref)
    runningControlCost = CostModelQuadratic(robot_sys, None, R)
    runningEECost = CostModelQuadraticTranslation(robot_sys,W_cur,p_ref_cur)

    if use_collision:
        runningCost = CostModelSum(robot_sys, [runningStateCost, runningControlCost, cost_collision, runningEECost])
    else:
        runningCost = CostModelSum(robot_sys, [runningStateCost, runningControlCost, runningEECost])        
    costs += [runningCost]

#### Setup crocoddyl model
rmodels = []
for i in range(T):
    state = crocoddyl.StateVector(robot_sys.Dx)
    rmodel = ActionModelRobot(state, robot_sys.Du)
    rmodel.init_robot_sys(robot_sys, nr = robot_sys.Dx)
    rmodel.set_cost(costs[i])
    rmodels += [rmodel]

terminalEECost = CostModelQuadraticTranslation(robot_sys,WT,p_ref)
terminalStateCost = CostModelQuadratic(robot_sys,Qf, x_ref = x_ref)
terminalControlCost = CostModelQuadratic(robot_sys, None,R)
if use_collision:
    terminalCost = CostModelSum(robot_sys, [terminalStateCost, terminalControlCost, cost_collision, terminalEECost])
else:
    terminalCost = CostModelSum(robot_sys, [terminalStateCost, terminalControlCost, terminalEECost])

rmodel_T = ActionModelRobot(state, robot_sys.Du)
rmodel_T.init_robot_sys(robot_sys, nr = robot_sys.Dx)
rmodel_T.set_cost(terminalCost)

#### Setup problem & initial guess

In [18]:
#initial state
if sys_name == 'quadcopter':
    x0 = np.array([ 0.,  0., 2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
    us = np.zeros((T, robot_sys.Du))
elif sys_name == 'bicopter':
    x0 = np.zeros(6)
    u_shift = 9.8*robot_sys.m/2
    us = np.ones((T, robot_sys.Du))*u_shift
elif sys_name == 'twolinkrobot':
    x0 = np.array([ 0.,  0., 0.,  0.])
    us = np.zeros((T, robot_sys.Du))
elif sys_name == 'manipulator':
    #x0 = np.zeros(robot_sys.Dx)
    us = np.zeros((T, robot_sys.Du))

    
robot_sys.set_init_state(x0)
robot_sys.compute_matrices(x0, us[0])
xs = robot_sys.rollout(us)

problem = crocoddyl.ShootingProblem(x0, rmodels, rmodel_T)

#### Solve

In [19]:
ddp = crocoddyl.SolverFDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackVerbose()])
ddp.solve(list(xs[:,:,None]), list(us[:,:,None]), maxiter=1000)

xs_ref, us_ref = np.array(ddp.xs), np.array(ddp.us)

In [20]:
traj = xs_ref[:,:7]
vis_traj_std(traj, dt = 0.03)

#### Visualize the path by multiple robots

#### For report: show the trajectory

#### Copy the data from ddp to ilqr solutions

In [21]:
ilqr = ILQR_Standard(robot_sys)
ilqr = get_ilqr_from_ddp(ddp,ilqr)

#### Use Least Square to compute the distribution

In [22]:
n_samples = 200
_ = ilqr.compute_du_LS()
sample_dxs, sample_dus = ilqr.sample_du(n_samples)

sample_dxs = np.array(sample_dxs).reshape(n_samples, T+1, -1)

sampling


#### Plot the cross-section distribution

#### Plot for each dimension

In [23]:
xs_sample = xs_ref + sample_dxs[np.random.randint(n_samples)]
vis_traj_std(xs_sample[:,:7], dt = 0.01)

In [24]:
cost_ori = ddp.problem.calc(list(xs_ref), list(us_ref))

# 2. Tracking the distribution with various strategies

## Defining the short time horizon ilqr

In [25]:
#### Setting cost
costs = []
Qf = np.eye(robot_sys.Dx)
for i in range(T_short+1):
    runningStateCostShort = CostModelQuadratic(robot_sys, Qf, x_ref = x_ref.copy())
    runningControlCostShort = CostModelQuadratic(robot_sys, None, R)
    if use_collision:
        runningCostShort = CostModelSum(robot_sys, [runningStateCostShort, runningControlCostShort, cost_collision])
    else:
        runningCostShort = CostModelSum(robot_sys, [runningStateCostShort, runningControlCostShort])        
    costs += [runningCostShort]

#### Setup crocoddyl model
rmodels = []
for i in range(T_short+1):
    stateShort = crocoddyl.StateVector(robot_sys.Dx)
    rmodelShort = ActionModelRobot(stateShort, robot_sys.Du)
    rmodelShort.init_robot_sys(robot_sys, nr = robot_sys.Dx)
    rmodelShort.set_cost(costs[i])
    rmodels += [rmodelShort]

#### Setup problem & initial guess

In [26]:
x0 = xs_ref[0]
robot_sys.set_init_state(x0)
xs = robot_sys.rollout(us)
problemShort = crocoddyl.ShootingProblem(x0, rmodels[:-1], rmodels[-1])
ddpShort = crocoddyl.SolverFDDP(problemShort)

# 2. Tracking the distribution with various strategies

## Defining the short time horizon ilqr

In [27]:
#### Setting cost
costs = []
for i in range(T_short+1):
    runningStateCostShort = CostModelQuadratic(robot_sys, Qf, x_ref = x_ref.copy())
    runningControlCostShort = CostModelQuadratic(robot_sys, None, R)
    if use_collision:
        runningCostShort = CostModelSum(robot_sys, [runningStateCostShort, runningControlCostShort, cost_collision])
    else:
        runningCostShort = CostModelSum(robot_sys, [runningStateCostShort, runningControlCostShort])        
    costs += [runningCostShort]

#### Setup crocoddyl model
rmodels = []
for i in range(T_short+1):
    stateShort = crocoddyl.StateVector(robot_sys.Dx)
    rmodelShort = ActionModelRobot(stateShort, robot_sys.Du)
    rmodelShort.init_robot_sys(robot_sys, nr = robot_sys.Dx)
    rmodelShort.set_cost(costs[i])
    rmodels += [rmodelShort]
    
reg_marg = 1e-2
reg_cond = 1e-2

#### Setup problem & initial guess

In [28]:
x0 = xs_ref[0]
robot_sys.set_init_state(x0)
xs = robot_sys.rollout(us)
problemShort = crocoddyl.ShootingProblem(x0, rmodels[:-1], rmodels[-1])
ddpShort = crocoddyl.SolverFDDP(problemShort)

#### Define the noises

In [39]:
if sys_name == 'quadcopter':
    noise_amp = 2.
    noise_v = normalize(np.random.rand(robot_sys.Dx/2)-0.5)*noise_amp
    noise = np.concatenate([np.zeros(3), noise_v[:3], np.zeros(3), noise_v[3:]])
    #noise = np.zeros(12)
    #noise[10] = -.5
    start_t = 65
    end_t = 67
elif sys_name == 'manipulator':
    noise_amp = 1.
    noise_v = normalize(np.random.rand(robot_sys.Dx/2)-0.5)*noise_amp
    noise = np.concatenate([np.zeros(robot_sys.Dx/2), noise_v])
    start_t = 40
    end_t = 42
    
#set the timing of the noise
noises = np.zeros((T, robot_sys.Dx))
for t in range(start_t, end_t):
    noises[t] = noise
print(noise)

[ 0.     0.     0.     0.     0.     0.     0.     0.248  0.387  0.505
 -0.471  0.09   0.268 -0.481]


### 2.1. MPC Strategy: following the mean

In [40]:
Qs_mean = np.tile(Qf, (T+T_short+1,1,1))

#initialise x
x = x0.copy()

#set the output containers
xs_mean = [x]
us_mean = []
x_ref_mean_set = [] #ref at each iter
x_output_mean_set = [] #output of ddp at each iter.

#set initial guess
xs_init = xs_ref[:T_short+1]
us_init = us_ref[:T_short]
xs_init[0] = x.copy()

for t in range(T):
    print(t)
    
    #obtain and set the reference traj
    x_ref = subsample(xs_ref[t:t+T_short+1], T_short+1)
    set_ref(ddpShort, x_ref)
    Qs_ref = Qs_mean[t:t+T_short+1]
    set_Qref(ddpShort, Qs_ref)
    #modify the ddp initial point 
    ddpShort.problem.x0 = x
    ddpShort.x0 = x
    
    #solve the ddp
    ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
    clear_output()
    
    #take the first control and execute (with noise)
    u = ddpShort.us[0]
    x = robot_sys.step(x, u) + noises[t]

    #store the data
    xs_mean += [x]
    us_mean += [u]
    x_ref_mean_set += [x_ref]
    x_output_mean_set += [np.array(ddpShort.xs)]
    
    #initialising the next step with the previous solutions
    xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
    us_init = subsample(np.array(ddpShort.us), T_short)
    xs_init[0] = x.copy()

xs_mean = np.array(xs_mean)
us_mean = np.array(us_mean)
cost_mean = ddp.problem.calc(list(xs_mean), list(us_mean))
print('The cost of tracking the mean is {}'.format(cost_mean))

The cost of tracking the mean is 51.1815261514


### 2.2. MPC Strategy: ILQR Feedback

In [41]:
#initialise x
x = x0.copy()

#setup the output container
xs_res_feed = [x]
us_res_feed = []

for t in range(T):
    print(t)
    u = ddp.us[t] - ddp.K[t].dot(x - ddp.xs[t])
    x = robot_sys.step(x, u) + noises[t]
    xs_res_feed += [x]
    us_res_feed += [u]
    
xs_res_feed = np.array(xs_res_feed)
us_res_feed = np.array(us_res_feed)
cost_feed = ddp.problem.calc(list(xs_res_feed), list(us_res_feed))
clear_output()
print('The cost of ilqr feedback control is is {}'.format(cost_feed))

The cost of ilqr feedback control is is 50.1071833157


### 2.3. MPC Strategy: Tracking marginal distribution

#### Compute marginal distribution

In [42]:
_, Qs = extract_ref(xs_ref[1:], ilqr.Sigma_x[robot_sys.Dx:, robot_sys.Dx:], robot_sys.Dx, T+T_short, xs_ref[0], reg_marg)

#### Tracking

In [43]:
#setup the initial state
x = x0.copy()

#setup the output containers
xs_res_dist = [x]
us_res_dist = []
x_ref_dist_set = []
x_output_dist_set = []

#setup the initial guess
xs_init = xs_ref[:T_short+1]
us_init = us_ref[:T_short]
xs_init[0] = x.copy()

for t in range(T):
    print(t)
    
    #extract the reference trajectory dist.
    x_ref = subsample(xs_ref[t:t+T_short+1], T_short+1)
    Qs_ref = Qs[t:t+T_short+1]
    set_ref(ddpShort, x_ref)
    set_Qref(ddpShort, Qs_ref)
    
    #modify the initial state of ddp
    ddpShort.problem.x0 = x
    ddpShort.x0 = x
    
    #solve
    ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
    clear_output()
    
    #take the first control and execute
    u = ddpShort.us[0]
    x = robot_sys.step(x, u) + noises[t]

    #store the output
    xs_res_dist += [x]
    us_res_dist += [u]
    x_ref_dist_set += [x_ref]
    x_output_dist_set += [np.array(ddpShort.xs)]
    
    #set the initial guess from the previous solution
    xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
    us_init = subsample(np.array(ddpShort.us), T_short)
    xs_init[0] = x.copy()

xs_res_dist = np.array(xs_res_dist)
us_res_dist = np.array(us_res_dist)
cost_dist = ddp.problem.calc(list(xs_res_dist), list(us_res_dist))
clear_output()
print('The cost of tracking the marginal distribution is {}'.format(cost_dist))

The cost of tracking the marginal distribution is 51.2546908551


### 2.4. MPC Strategy:  tracking conditional distribution

### Setup the trajectory distribution

In [44]:
#the full trajectory distribution
x_dist = Gaussian()
x_dist.D = robot_sys.Dx*(T+1+T_short)
x_dist.mu = ilqr.xs.flatten()
x_dist.sigma = np.zeros((ilqr.Sigma_x.shape[0]+T_short*robot_sys.Dx, ilqr.Sigma_x.shape[1]+T_short*robot_sys.Dx))
x_dist.sigma[:ilqr.Sigma_x.shape[0],:ilqr.Sigma_x.shape[0]] = ilqr.Sigma_x + 1e-6*np.eye(ilqr.Sigma_x.shape[0])
for t in range(T+1, T+1+T_short):
    x_dist.sigma[t*robot_sys.Dx:(t+1)*robot_sys.Dx, t*robot_sys.Dx:(t+1)*robot_sys.Dx] = x_dist.sigma[T*robot_sys.Dx:(T+1)*robot_sys.Dx, T*robot_sys.Dx:(T+1)*robot_sys.Dx]
    x_dist.mu = np.concatenate([x_dist.mu, xs_ref[-1]])
    #the conditional distribution along the time horizon T_short
x_dist_short = Gaussian()
x_dist_short.D = robot_sys.Dx*(T_short+1)

#### Tracking

In [45]:
#set initial state
x = x0.copy()

#set the output container
xs_res_cond = [x]
us_res_cond = []
x_ref_cond_set = []
x_output_cond_set = []

#set the initial guess
xs_init = xs_ref[:T_short+1]
us_init = us_ref[:T_short]
xs_init[0] = x.copy()

for t in range(T):
    print(t)
    
    mu,sigma = x_dist.get_marginal(slice(t*robot_sys.Dx, (t+1+T_short)*robot_sys.Dx))
    x_dist_short.mu = mu
    x_dist_short.sigma = sigma

    #get conditional distribution
    mu,sigma = x_dist_short.condition(x, slice(0,robot_sys.Dx), slice(robot_sys.Dx, x_dist.mu.shape[0]))
    x_dist_short.mu = mu
    x_dist_short.sigma = sigma

    #obtain references
    x_ref, Qs_ref = extract_ref(x_dist_short.mu, x_dist_short.sigma, robot_sys.Dx, T_short, x, reg = 1e-2)

    #set the reference
    set_ref(ddpShort, x_ref)
    set_Qref(ddpShort, 1*Qs_ref)
        
    #modify ddp initial state
    ddpShort.problem.x0 = x
    ddpShort.x0 = x
    
    #solve
    ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
    clear_output()
    
    #take the first control and execute
    u = ddpShort.us[0]
    x = robot_sys.step(x, u) + noises[t]
    
    #store the output into the containers
    xs_res_cond += [x]
    us_res_cond += [u]
    x_ref_cond_set += [x_ref]
    x_output_cond_set += [np.array(ddpShort.xs)]

    #set the initial guess
    xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
    us_init = subsample(np.array(ddpShort.us), T_short)
    xs_init[0] = x.copy()


xs_res_cond = np.array(xs_res_cond)
us_res_cond = np.array(us_res_cond)
cost_cond = ddp.problem.calc(list(xs_res_cond), list(us_res_cond))
clear_output()
print('The cost of tracking the conditional distribution is {}'.format(cost_cond))

The cost of tracking the conditional distribution is 50.6167696568


#### Comparison

In [46]:
print('Feedback: {}, Mean: {}, Dist: {}, Cond: {}'.format(cost_feed, cost_mean, cost_dist, cost_cond))

Feedback: 50.1071833157, Mean: 51.1815261514, Dist: 51.2546908551, Cond: 50.6167696568


In [47]:
print('cost_state, cost_control, cost_col,  cost_goal')
cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_feed, us_res_feed, ddp)
print('feed', cost_state, cost_control, np.sum(cost_col), cost_goal)
cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_mean, us_mean, ddp)
print('mean', cost_state, cost_control, np.sum(cost_col), cost_goal)
cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_dist, us_res_dist, ddp)
print('dist', cost_state, cost_control, np.sum(cost_col), cost_goal)
cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_cond, us_res_cond, ddp)
print('cond', cost_state, cost_control, np.sum(cost_col), cost_goal)


cost_state, cost_control, cost_col,  cost_goal
('feed', 14.708824234609928, 2.295088758866839, 1.6333302713310933, 8.079826217649359)
('mean', 14.48344423044769, 2.6195336566815186, 0.4546595316785199, 9.26968782267232)
('dist', 13.0410589238442, 2.7238088785616457, 1.2567800471585262, 10.56836142039978)
('cond', 13.89317091390972, 2.1945432274194614, 1.0850320827844118, 9.58563309167752)


***

# Batch comparison

In [68]:
import time
tic = time.time()
n_test = 10

to_build_from_zero = False
noise_sizes = ['small', 'medium', 'large']
method_names = ['feed', 'mean', 'marg', 'cond']
field_names = ['xs', 'us', 'x_ref_set', 'x_output_set', 'cost', 'noise', 
               'cost_state', 'cost_col', 'cost_goal', 'cost_control']
if to_build_from_zero:
    data = {'small':dict(), 'medium':dict(), 'large':dict()}
    for noise_size in noise_sizes:
        for method_name in method_names:
            data[noise_size][method_name] = dict()
            for field_name in field_names:
                data[noise_size][method_name][field_name] = []
else:
    data = np.load('data/ilqr_'+sys_name+'2.npy', allow_pickle = True)[()]
    for noise_size in noise_sizes:
        for method_name in method_names:
            for field_name in field_names:
                data[noise_size][method_name][field_name] = list(data[noise_size][method_name][field_name])

In [69]:
for noise_size in noise_sizes[:2]:
#     if noise_size == 'medium':
#         start_k = 16
#     else:
#         start_k = 0
    start_k = 0
    for k in range(start_k, n_test):
        #### Define the noises
        if sys_name == 'quadcopter':           
            if noise_size == 'large':
                noise_amp = 1.2 #3.
            elif noise_size == 'medium':
                noise_amp = 0.7#1.5
            elif noise_size == 'small':
                noise_amp = 0.2#0.5
            noise_v = normalize(np.random.rand(robot_sys.Dx/2)-0.5)*noise_amp
            noise = np.concatenate([np.zeros(3), noise_v[:3], np.zeros(3), noise_v[3:]])
            #noise = np.zeros(12)
            #noise[10] = -.5
            start_t = 65
            end_t = 67
        elif sys_name == 'manipulator':
            if noise_size == 'large':
                noise_amp = 3.
            elif noise_size == 'medium':
                noise_amp = 1.5
            elif noise_size == 'small':
                noise_amp = 0.5
            noise_v = normalize(np.random.rand(robot_sys.Dx/2)-0.5)*noise_amp
            noise = np.concatenate([np.zeros(robot_sys.Dx/2), noise_v])
            start_t = 40
            end_t = 42

        #set the timing of the noise
        noises = np.zeros((T, robot_sys.Dx))
        for t in range(start_t, end_t):
            noises[t] = noise

        ### 2.1. MPC Strategy: following the mean
        Qs_mean = np.tile(Qf, (T+T_short+1,1,1))

        #initialise x
        x = x0.copy()

        #set the output containers
        xs_mean = [x]
        us_mean = []
        x_ref_mean_set = [] #ref at each iter
        x_output_mean_set = [] #output of ddp at each iter.

        #set initial guess
        xs_init = xs_ref[:T_short+1]
        us_init = us_ref[:T_short]
        xs_init[0] = x.copy()

        for t in range(T):
            print(t)

            #obtain and set the reference traj
            x_ref = subsample(xs_ref[t:t+T_short+1], T_short+1)
            set_ref(ddpShort, x_ref)
            Qs_ref = Qs_mean[t:t+T_short+1]
            set_Qref(ddpShort, Qs_ref)
            #modify the ddp initial point 
            ddpShort.problem.x0 = x
            ddpShort.x0 = x

            #solve the ddp
            ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
            clear_output()

            #take the first control and execute (with noise)
            u = ddpShort.us[0]
            x = robot_sys.step(x, u) + noises[t]

            #store the data
            xs_mean += [x]
            us_mean += [u]
            x_ref_mean_set += [x_ref]
            x_output_mean_set += [np.array(ddpShort.xs)]

            #initialising the next step with the previous solutions
            xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
            us_init = subsample(np.array(ddpShort.us), T_short)
            xs_init[0] = x.copy()

        xs_mean = np.array(xs_mean)
        us_mean = np.array(us_mean)
        cost_mean = ddp.problem.calc(list(xs_mean), list(us_mean))
        print('The cost of tracking the mean is {}'.format(cost_mean))


        ### 2.2. MPC Strategy: ILQR Feedback

        #initialise x
        x = x0.copy()

        #setup the output container
        xs_res_feed = [x]
        us_res_feed = []

        for t in range(T):
            print(t)
            u = ddp.us[t] - ddp.K[t].dot(x - ddp.xs[t])
            x = robot_sys.step(x, u) + noises[t]
            xs_res_feed += [x]
            us_res_feed += [u]

        xs_res_feed = np.array(xs_res_feed)
        us_res_feed = np.array(us_res_feed)
        cost_feed = ddp.problem.calc(list(xs_res_feed), list(us_res_feed))
        clear_output()
        print('The cost of ilqr feedback control is is {}'.format(cost_feed))


        ### 2.3. MPC Strategy: Tracking marginal distribution

        #### Compute marginal distribution
        _, Qs = extract_ref(xs_ref[1:], ilqr.Sigma_x[robot_sys.Dx:, robot_sys.Dx:], 
                            robot_sys.Dx, T+T_short, xs_ref[0], reg_marg)

        #### Tracking

        #setup the initial state
        x = x0.copy()

        #setup the output containers
        xs_res_dist = [x]
        us_res_dist = []
        x_ref_dist_set = []
        x_output_dist_set = []

        #setup the initial guess
        xs_init = xs_ref[:T_short+1]
        us_init = us_ref[:T_short]
        xs_init[0] = x.copy()

        for t in range(T):
            print(t)

            #extract the reference trajectory dist.
            x_ref = subsample(xs_ref[t:t+T_short+1], T_short+1)
            Qs_ref = Qs[t:t+T_short+1]
            set_ref(ddpShort, x_ref)
            set_Qref(ddpShort, Qs_ref)

            #modify the initial state of ddp
            ddpShort.problem.x0 = x
            ddpShort.x0 = x

            #solve
            ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
            clear_output()

            #take the first control and execute
            u = ddpShort.us[0]
            x = robot_sys.step(x, u) + noises[t]

            #store the output
            xs_res_dist += [x]
            us_res_dist += [u]
            x_ref_dist_set += [x_ref]
            x_output_dist_set += [np.array(ddpShort.xs)]

            #set the initial guess from the previous solution
            xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
            us_init = subsample(np.array(ddpShort.us), T_short)
            xs_init[0] = x.copy()

        xs_res_dist = np.array(xs_res_dist)
        us_res_dist = np.array(us_res_dist)
        cost_dist = ddp.problem.calc(list(xs_res_dist), list(us_res_dist))
        clear_output()
        print('The cost of tracking the marginal distribution is {}'.format(cost_dist))


        ### 2.4. MPC Strategy:  tracking conditional distribution

        ### Setup the trajectory distribution

        #the full trajectory distribution
        x_dist = Gaussian()
        x_dist.D = robot_sys.Dx*(T+1+T_short)
        x_dist.mu = ilqr.xs.flatten()
        x_dist.sigma = np.zeros((ilqr.Sigma_x.shape[0]+T_short*robot_sys.Dx, ilqr.Sigma_x.shape[1]+T_short*robot_sys.Dx))
        x_dist.sigma[:ilqr.Sigma_x.shape[0],:ilqr.Sigma_x.shape[0]] = ilqr.Sigma_x + 1e-6*np.eye(ilqr.Sigma_x.shape[0])
        for t in range(T+1, T+1+T_short):
            x_dist.sigma[t*robot_sys.Dx:(t+1)*robot_sys.Dx, t*robot_sys.Dx:(t+1)*robot_sys.Dx] = x_dist.sigma[T*robot_sys.Dx:(T+1)*robot_sys.Dx, T*robot_sys.Dx:(T+1)*robot_sys.Dx]
            x_dist.mu = np.concatenate([x_dist.mu, xs_ref[-1]])
            #the conditional distribution along the time horizon T_short
        x_dist_short = Gaussian()
        x_dist_short.D = robot_sys.Dx*(T_short+1)

        #### Tracking

        #set initial state
        x = x0.copy()

        #set the output container
        xs_res_cond = [x]
        us_res_cond = []
        x_ref_cond_set = []
        x_output_cond_set = []

        #set the initial guess
        xs_init = xs_ref[:T_short+1]
        us_init = us_ref[:T_short]
        xs_init[0] = x.copy()

        for t in range(T):
            print(t)

            mu,sigma = x_dist.get_marginal(slice(t*robot_sys.Dx, (t+1+T_short)*robot_sys.Dx))
            x_dist_short.mu = mu
            x_dist_short.sigma = sigma

            #get conditional distribution
            mu,sigma = x_dist_short.condition(x, slice(0,robot_sys.Dx), slice(robot_sys.Dx, x_dist.mu.shape[0]))
            x_dist_short.mu = mu
            x_dist_short.sigma = sigma

            #obtain references
            x_ref, Qs_ref = extract_ref(x_dist_short.mu, x_dist_short.sigma, robot_sys.Dx, T_short, x, reg = 1e-2)

            #set the reference
            set_ref(ddpShort, x_ref)
            set_Qref(ddpShort, 1*Qs_ref)

            #modify ddp initial state
            ddpShort.problem.x0 = x
            ddpShort.x0 = x

            #solve
            ddpShort.solve(list(xs_init), list(us_init), n_iter_short)
            clear_output()

            #take the first control and execute
            u = ddpShort.us[0]
            x = robot_sys.step(x, u) + noises[t]

            #store the output into the containers
            xs_res_cond += [x]
            us_res_cond += [u]
            x_ref_cond_set += [x_ref]
            x_output_cond_set += [np.array(ddpShort.xs)]

            #set the initial guess
            xs_init = subsample(np.array(ddpShort.xs[1:]), T_short+1)
            us_init = subsample(np.array(ddpShort.us), T_short)
            xs_init[0] = x.copy()


        xs_res_cond = np.array(xs_res_cond)
        us_res_cond = np.array(us_res_cond)
        cost_cond = ddp.problem.calc(list(xs_res_cond), list(us_res_cond))
        clear_output()
        print('The cost of tracking the conditional distribution is {}'.format(cost_cond))


        #### Comparison
        print('Feedback: {}, Mean: {}, Dist: {}, Cond: {}'.format(cost_feed, cost_mean, cost_dist, cost_cond))


        ####store the data
        #data mean
        cost_state, cost_control, cost_col, cost_goal  = calc_detail_cost(xs_mean, us_mean, ddp)
        data[noise_size]['mean']['xs'] += [xs_mean]
        data[noise_size]['mean']['us'] += [us_mean]
        data[noise_size]['mean']['x_ref_set'] += [x_ref_mean_set]
        data[noise_size]['mean']['x_output_set'] += [x_output_mean_set]
        data[noise_size]['mean']['cost'] += [cost_mean]
        data[noise_size]['mean']['noise'] += [noise]
        data[noise_size]['mean']['cost_state'] += [cost_state]
        data[noise_size]['mean']['cost_control'] += [cost_control]
        data[noise_size]['mean']['cost_col'] += [cost_col]
        data[noise_size]['mean']['cost_goal'] += [cost_goal]

        #data feedback
        cost_state, cost_control, cost_col, cost_goal  = calc_detail_cost(xs_res_feed, us_res_feed, ddp)
        data[noise_size]['feed']['xs'] += [xs_res_feed]
        data[noise_size]['feed']['us'] += [us_res_feed]
        data[noise_size]['feed']['cost'] += [cost_feed]
        data[noise_size]['feed']['noise'] += [noise]
        data[noise_size]['feed']['cost_state'] += [cost_state]
        data[noise_size]['feed']['cost_control'] += [cost_control]
        data[noise_size]['feed']['cost_col'] += [cost_col]
        data[noise_size]['feed']['cost_goal'] += [cost_goal]


        #data margin
        cost_state, cost_control, cost_col, cost_goal  = calc_detail_cost(xs_res_dist, us_res_dist, ddp)
        data[noise_size]['marg']['xs'] += [xs_res_dist]
        data[noise_size]['marg']['us'] += [us_res_dist]
        data[noise_size]['marg']['x_ref_set'] += [x_ref_dist_set]
        data[noise_size]['marg']['x_output_set'] += [x_output_dist_set]
        data[noise_size]['marg']['cost'] += [cost_dist]
        data[noise_size]['marg']['noise'] += [noise]
        data[noise_size]['marg']['cost_state'] += [cost_state]
        data[noise_size]['marg']['cost_control'] += [cost_control]
        data[noise_size]['marg']['cost_col'] += [cost_col]
        data[noise_size]['marg']['cost_goal'] += [cost_goal]

        #data 
        cost_state, cost_control, cost_col, cost_goal  = calc_detail_cost(xs_res_cond, us_res_cond, ddp)
        data[noise_size]['cond']['xs'] += [xs_res_cond]
        data[noise_size]['cond']['us'] += [us_res_cond]
        data[noise_size]['cond']['x_ref_set'] += [x_ref_cond_set]
        data[noise_size]['cond']['x_output_set'] += [x_output_cond_set]
        data[noise_size]['cond']['cost'] += [cost_cond]
        data[noise_size]['cond']['noise'] += [noise]
        data[noise_size]['cond']['cost_state'] += [cost_state]
        data[noise_size]['cond']['cost_control'] += [cost_control]
        data[noise_size]['cond']['cost_col'] += [cost_col]
        data[noise_size]['cond']['cost_goal'] += [cost_goal]

        print('Feedback: {}, Mean: {}, Dist: {}, Cond: {}'.format(cost_feed, cost_mean, cost_dist, cost_cond))
        print('cost_state, cost_control, cost_col,  cost_goal')
        cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_feed, us_res_feed, ddp)
        print('feed', cost_state, cost_control, np.sum(cost_col), cost_goal)
        cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_mean, us_mean, ddp)
        print('mean', cost_state, cost_control, np.sum(cost_col), cost_goal)
        cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_dist, us_res_dist, ddp)
        print('dist', cost_state, cost_control, np.sum(cost_col), cost_goal)
        cost_state, cost_control, cost_col, cost_goal = calc_detail_cost(xs_res_cond, us_res_cond, ddp)
        print('cond', cost_state, cost_control, np.sum(cost_col), cost_goal)

The cost of tracking the conditional distribution is 51.8731414367
Feedback: 51.3177438573, Mean: 52.829156864, Dist: 52.9946996015, Cond: 51.8731414367
Feedback: 51.3177438573, Mean: 52.829156864, Dist: 52.9946996015, Cond: 51.8731414367
cost_state, cost_control, cost_col,  cost_goal
('feed', 16.217819393609776, 2.9329425937359748, 1.6916636529198918, 8.0794409495046)
('mean', 15.144486382462537, 3.9123873519664403, 0.6369503516740158, 9.131631621111053)
('dist', 13.577072138308699, 4.193000663931512, 1.263164092087105, 10.568008587948807)
('cond', 15.588677883329945, 2.814413677559748, 1.1409144983110768, 9.583476181975248)


In [70]:
for noise_size in noise_sizes:
    for method_name in method_names:
        for field_name in field_names:
            data[noise_size][method_name][field_name] = np.array(data[noise_size][method_name][field_name])
    

toc = time.time()
print('Computation takes {}'.format(toc-tic))

Computation takes 865.959942102


In [71]:
np.save('data/ilqr_'+sys_name+'2.npy', data)

***